<a href="https://colab.research.google.com/github/Shunian-Chen/AIPI530/blob/Assignment4/Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline


# Assignment 4


In [2]:
!/opt/bin/nvidia-smi

Mon Nov 15 20:41:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
PATH = "/content/drive/Othercomputers/我的笔记本电脑/Google Drive/A4"
os.chdir(PATH)
os.getcwd()

'/content/drive/Othercomputers/我的笔记本电脑/Google Drive/A4'

## Import libraries

In [5]:
!pip install gym-super-mario-bros==7.3.0
!pip install git+https://github.com/Shunian-Chen/AIPI530.git@Assignment4
!apt-get install ffmpeg freeglut3-dev xvfb  # For visualization


import torch
from torch import nn
from torchvision import transforms as T
from PIL import Image
import numpy as np
from pathlib import Path
from collections import deque
import random, datetime, os, copy

# Gym is an OpenAI toolkit for RL
import gym
from gym.spaces import Box
from gym.wrappers import FrameStack

# NES Emulator for OpenAI Gym
from nes_py.wrappers import JoypadSpace

# Super Mario environment for OpenAI Gym
import gym_super_mario_bros

import gym
import numpy as np
import torch as th
import matplotlib.pyplot as plt

#Import DQN and evaluation helper
from stable_baselines3 import DQN 
from stable_baselines3.common.evaluation import evaluate_policy

     |████████████████████████████████| 198 kB 4.2 MB/s 
     |████████████████████████████████| 76 kB 6.3 MB/s 
  Created wheel for nes-py: filename=nes_py-8.1.8-cp37-cp37m-linux_x86_64.whl size=437385 sha256=4f1c3f23ae61dcaa30febf61a708b1cbad4259acb5edd94a60110581ee6ae459
  Stored in directory: /root/.cache/pip/wheels/f2/05/1f/608f15ab43187096eb5f3087506419c2d9772e97000f3ba025
Successfully built nes-py
  Cloning https://github.com/Shunian-Chen/AIPI530.git (to revision Assignment4) to /tmp/pip-req-build-6ymyrcb5
  Running command git clone -q https://github.com/Shunian-Chen/AIPI530.git /tmp/pip-req-build-6ymyrcb5
  Running command git checkout -b Assignment4 --track origin/Assignment4
  Switched to a new branch 'Assignment4'
  Branch 'Assignment4' set up to track remote branch 'Assignment4' from 'origin'.
  Created wheel for stable-baselines3: filename=stable_baselines3-1.3.1a1-py3-none-any.whl size=163781 sha256=951c6e12907d318339293df962980c5d4dd6d8afb3bf1d1adbc4558fc57e3c75
  Store

## Preprocess Environment



In [6]:
class SkipFrame(gym.Wrapper):
    def __init__(self, env, skip):
        """Return only every `skip`-th frame"""
        super().__init__(env)
        self._skip = skip

    def step(self, action):
        """Repeat action, and sum reward"""
        total_reward = 0.0
        done = False
        for i in range(self._skip):
            # Accumulate reward and repeat the same action
            obs, reward, done, info = self.env.step(action)
            total_reward += reward
            if done:
                break
        return obs, total_reward, done, info


class GrayScaleObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        obs_shape = self.observation_space.shape[:2]
        self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)

    def permute_orientation(self, observation):
        # permute [H, W, C] array to [C, H, W] tensor
        observation = np.transpose(observation, (2, 0, 1))
        observation = torch.tensor(observation.copy(), dtype=torch.float)
        return observation

    def observation(self, observation):
        observation = self.permute_orientation(observation)
        transform = T.Grayscale()
        observation = transform(observation)
        return observation


class ResizeObservation(gym.ObservationWrapper):
    def __init__(self, env, shape):
        super().__init__(env)
        if isinstance(shape, int):
            self.shape = (shape, shape)
        else:
            self.shape = tuple(shape)

        obs_shape = self.shape + self.observation_space.shape[2:]
        self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)

    def observation(self, observation):
        transforms = T.Compose(
            [T.Resize(self.shape), T.Normalize(0, 255)]
        )
        observation = transforms(observation).squeeze(0)
        return observation

def prepare_env():
  # Initialize Super Mario environment
  env = gym_super_mario_bros.make("SuperMarioBros-1-1-v0")

  # Limit the action-space to
  #   0. walk right
  #   1. jump right
  env = JoypadSpace(env, [["right"], ["right", "A"]])

  env.reset()

  next_state, reward, done, info = env.step(action=0)
  print(f"{next_state.shape},\n {reward},\n {done},\n {info}")

  # Apply Wrappers to environment
  env = SkipFrame(env, skip=4)
  env = GrayScaleObservation(env)
  env = ResizeObservation(env, shape=84)
  env = FrameStack(env, num_stack=4)

  return env


In [7]:
env = prepare_env()

(240, 256, 3),
 0,
 False,
 {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 400, 'world': 1, 'x_pos': 40, 'x_pos_screen': 40, 'y_pos': 79}


## Model Training

In [8]:
tensorboard_log = "data/tb/"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_freq=16
epoch = int(5e4) * train_freq
save_period = int(1e3) * train_freq
save_path = "dqn_model"

try:
  env = prepare_env()
  dqn_model = DQN.load(save_path + "_model", env = env)
  dqn_model.load_replay_buffer(save_path + "_replay_buffer.pkl")
except:
  dqn_model = DQN("CnnPolicy",
              env,
              verbose=1,
              train_freq=16,
              gradient_steps=8,
              gamma=0.99,
              exploration_fraction=0.4,
              exploration_final_eps=0.1,
              target_update_interval=600,
              learning_starts=1000,
              buffer_size=10000,
              batch_size=256,
              learning_rate=4e-3,
              policy_kwargs=dict(net_arch=[256, 256]),
              tensorboard_log=tensorboard_log,
              seed=2,
              device = device)

(240, 256, 3),
 0,
 False,
 {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 400, 'world': 1, 'x_pos': 40, 'x_pos_screen': 40, 'y_pos': 79}
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [9]:
mean_reward, std_reward = evaluate_policy(dqn_model, dqn_model.get_env(), deterministic=True, n_eval_episodes=20)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

/usr/local/lib/python3.7/dist-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


mean_reward:231.00 +/- 0.00


I embeded the saving process inside the .learn() mothod, so that it will automatically save the model at "save_path" every "save_period"

In [ ]:
dqn_model.learn(total_timesteps=epoch, log_interval=20, save_period = save_period, save_path = save_path, reset_num_timesteps = False)

Logging to data/tb/DQN_31


/usr/local/lib/python3.7/dist-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 91       |
|    ep_rew_mean      | 415      |
|    exploration_rate | 0.105    |
| time/               |          |
|    episodes         | 4300     |
|    fps              | 17       |
|    time_elapsed     | 84       |
|    total_timesteps  | 527868   |
| train/              |          |
|    learning_rate    | 0.004    |
|    loss             | 2.73e+11 |
|    n_updates        | 263432   |
----------------------------------
Steps: 528000, use time: 89.29711961746216, average reward: 403.25!
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 84       |
|    ep_rew_mean      | 386      |
|    exploration_rate | 0.102    |
| time/               |          |
|    episodes         | 4320     |
|    fps              | 23       |
|    time_elapsed     | 130      |
|    total_timesteps  | 529416   |
| train/              |          |
|    learning_rate    

In [ ]:
mean_reward, std_reward = evaluate_policy(dqn_model, dqn_model.get_env(), deterministic=True, n_eval_episodes=20)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
#Monitor training in tensorboard
%load_ext tensorboard
%tensorboard --logdir $tensorboard_log

## Prepare video recording

In [ ]:
# Set up fake display; otherwise rendering will fail
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [ ]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [ ]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_id, model, video_length=5000, prefix='', video_folder='videos/'):
  """
  :param env_id: (str)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  eval_env = env_id
  # Start the video at step=0 and record 500 steps
  eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs, deterministic=False)
    obs, _, _, _ = eval_env.step(action)

  # Close the video recorder
  eval_env.close()

## Visualize trained agent

In [ ]:
record_video(dqn_model.get_env(), dqn_model, video_length=1000, prefix='dqn-mario-save')

In [ ]:
show_videos('videos', prefix='dqn-mario-save')